In [21]:
import pandas as pd
import numpy as np
from gurobipy import GRB
import gurobipy as gp

In [22]:
demand = [0,0,0,5000,6000,6500,7000,8000,9500]

In [23]:
m = gp.Model("Production")

In [24]:
## Allocate decision variables
x = m.addMVar(shape=(len(demand)), vtype=GRB.INTEGER , name="decision")

In [25]:
## Add constraints on max/min production values
m.addConstr(x[0] == 500)
m.addConstr(x[1] == 2000)
m.addConstr(x[2] == 1000)

for i in range(3, x.shape[0]):
    m.addConstr(
        0 <= x[i]
    )
    m.addConstr(
        x[i] <= 8000
    )

In [26]:
## Add intermediate variable y to represent stock allocations
## y[i][0] -> production for current month, y[i][1] -> residual from month i-1, y[i][2] -> residual from month i-2
y = m.addMVar(shape=(len(demand), 3), vtype=GRB.INTEGER , name="alloc")

for i in range(x.shape[0]):
    for j in range(3):
        m.addConstr(
            y[i][j] >= 0
            )

In [27]:
## Add basic constraints on allocations
for i in range(x.shape[0]):
    m.addConstr(
        x[i] == gp.quicksum(y[i][j] for j in range(3))
    )

In [28]:
## Add constracts to meet demand of each month
for i in range(3, x.shape[0]):
    m.addConstr(
        demand[i] <= y[i][0] + y[i-1][1]*0.89 + y[i-2][2]*0.47
    )

In [29]:
obj_f = sum([((y[i][1]*0.11)*25 + (y[i][2]*0.53)*25 + (y[i][1])*0.75 + (y[i][2])*0.75*2) for i in range(x.shape[0])])

In [30]:
m.setObjective(
    obj_f
    ,GRB.MINIMIZE
)

In [31]:
try: 
    m.optimize()
except gp.GurobiError as E:
    print("Optimize failed", E)

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[x86])
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 57 rows, 36 columns and 96 nonzeros
Model fingerprint: 0x13d8712e
Variable types: 0 continuous, 36 integer (0 binary)
Coefficient statistics:
  Matrix range     [5e-01, 1e+00]
  Objective range  [4e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+02, 1e+04]
Found heuristic solution: objective 76603.500000
Presolve removed 47 rows and 21 columns
Presolve time: 0.00s
Presolved: 10 rows, 15 columns, 29 nonzeros
Found heuristic solution: objective 52301.750000
Variable types: 0 continuous, 15 integer (0 binary)
Found heuristic solution: objective 19313.000000

Root relaxation: objective 1.604138e+04, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 16041.37

In [32]:
for i in range(x.shape[0]):
    print(x[i].x, y[i].x)

500.0 [500.  -0.  -0.]
2000.0 [2000.   -0.   -0.]
1000.0 [1000.   -0.   -0.]
8000.0 [8000.   -0.   -0.]
8000.0 [8000.   -0.   -0.]
8000.0 [6994. 1006.   -0.]
8000.0 [6105. 1895.   -0.]
8000.0 [6314. 1686.   -0.]
8000.0 [8000.   -0.   -0.]
